In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [2]:
'https://stackoverflow.com/questions/35792278/how-to-find-number-of-parameters-of-a-keras-model'
def size_model(model): # Compute number of params in a model (the actual number of floats)
    import numpy as np
    #return sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])
    W = [K.get_value(w).shape for w in model.trainable_weights]
    print(W)
    total = sum([np.prod(w) for w in W])
    return total

In [3]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 20 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## 首先我們使用一般的 DNN (MLP) 來訓練
由於 DNN 只能輸入一維的資料，我們要先將影像進行攤平，若 (50000, 32, 32, 3) 的影像，攤平後會變成 (50000, 32*32*3) = (50000, 3072)

In [4]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [5]:
model_dnn = Sequential()
model_dnn.add(Dense(512, activation='relu', input_shape=(3072,)))
model_dnn.add(Dropout(0.2))
model_dnn.add(Dense(512, activation='relu'))
model_dnn.add(Dropout(0.2))
model_dnn.add(Dense(num_classes, activation='softmax'))

model_dnn.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
___________

In [6]:
model_dnn.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model_dnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model_dnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 4s 78us/step - loss: 2.1401 - acc: 0.2524 - val_loss: 2.0492 - val_acc: 0.2851
Epoch 2/20
50000/50000 [==============================] - 2s 46us/step - loss: 1.8566 - acc: 0.3313 - val_loss: 1.7621 - val_acc: 0.3735
Epoch 3/20
50000/50000 [==============================] - 2s 46us/step - loss: 1.7765 - acc: 0.3624 - val_loss: 1.7261 - val_acc: 0.3803
Epoch 4/20
50000/50000 [==============================] - 2s 47us/step - loss: 1.7253 - acc: 0.3819 - val_loss: 1.6604 - val_acc: 0.4165
Epoch 5/20
50000/50000 [==============================] - 2s 48us/step - loss: 1.6973 - acc: 0.3896 - val_loss: 1.6130 - val_acc: 0.4304
Epoch 6/20
50000/50000 [==============================] - 2s 48us/step - loss: 1.6665 - acc: 0.4009 - val_loss: 1.6686 - val_acc: 0.4047
Epoch 7/20
50000/50000 [==============================] - 2s 45us/step - loss: 1

## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [7]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [8]:
model_cnn = Sequential()
model_cnn.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_cnn.add(Activation('relu'))
model_cnn.add(Conv2D(32, (3, 3)))
model_cnn.add(Activation('relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))

model_cnn.add(Conv2D(64, (3, 3), padding='same'))
model_cnn.add(Activation('relu'))
model_cnn.add(Conv2D(64, (3, 3)))
model_cnn.add(Activation('relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))

model_cnn.add(Flatten())
model_cnn.add(Dense(512))
model_cnn.add(Activation('relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(num_classes))
model_cnn.add(Activation('softmax'))
model_cnn.summary()

model_cnn.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model_cnn.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model_cnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

## 同樣運算 10 個 epochs，但 CNN 在 test data 的準確率顯著優於 DNN!

## 作業
# 1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
可調整參數, 至少有 batch_size, epoch, learning rate. 

調 epoch, accuracy 增加, DNN 上升 1.4%, CNN 上升 2% 
                   10 epochs            20 epochs
DNN Test loss:     1.5602067405700684   1.548732175064087
DNN Test accuracy: 0.4487               0.4501
CNN Test loss:     0.7365292016983033   0.6883784538269043
CNN Test accuracy: 0.7479               0.7679


# 2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?
DNN_Total params: 1,841,162
DNN_Trainable params: 1,841,162
CNN_Total params: 1,250,858
CNN_Trainable params: 1,250,858
由上可知, DNN 參數 1.8M 比 CNN 1.2M 多
由於 dnn 是全 fully connected layer 建構, 每層需要維持 in_num x out_num 的大量參數, 和矩陣運算. 而 CNN 有參數共享可減少參數的優點說法. 但古典影像辨識法觀知, 古典影像辨識本身就是設計各種不同的 convolution filters 將影像轉為可區分之特徵. CNN 設計者只是了解影像辨識的特性, 將辨識所需的 convolution filters 交由神經網路學習. 符合特性的網路結構自然比較優越. 